In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

star_rating = {'1': '.star-rating.One',
        '2': '.star-rating.Two',
        '3': '.star-rating.Three',
        '4': '.star-rating.Four',
        '5': '.star-rating.Five'
        }

def book_search_by_stars(stars=5,csv_name='books_omar.csv',save=True):
    '''
    ABLE TO FIND BOOKS BY STAR RATING AND SAVE A .CSV WITHIN RESULTS.
    RETURNS A PANDAS DATAFRAME 
    
    stars: number of stars. int
    csv_name: name of the csv file. string
    
    returns a dataframe
    '''
    # For storage data
    cols = ['Title', 'Stars', 'Price', 'Availability']
    df = pd.DataFrame(columns=cols)

    # For scraping
    base_url = 'http://books.toscrape.com/catalogue/page-{}.html'
    page_num = 0
    book_num = 0
    
    # Scraping
    while True:
        
        # Web setup
        page_num += 1
        rating = star_rating[str(stars)]
        res = requests.get(base_url.format(page_num))
        soup = BeautifulSoup(res.text,'lxml')
        books = soup.select('.product_pod')

        # If the page still availabe
        if soup.select('title')[0].text != '404 Not Found':
            for book in books:
                # If it has n stars
                if len(book.select(rating)) != 0:
                    book_num += 1
                    title = book.select('a')[1]['title']
                    price = book.select('.price_color')[0].text
                    if 'In stock' in book.select('.instock.availability')[0].text:
                        availability = 1
                    else:
                        availability = 0

                    df.loc[book_num] = (title,stars,price[2:],availability)
                    
        # When done            
        else:
            if save:
                df.to_csv(csv_name, index=False)
                
            return df

In [2]:
book_search_by_stars(stars=2)

,Title,Stars,Price,Availability
1,"Starving Hearts (Triangular Trade Trilogy, #1)",2,13.99,1
2,Libertarianism for Beginners,2,51.33,1
3,It's Only the Himalayas,2,45.17,1
4,How Music Works,2,37.32,1
5,Maude (1883-1993):She Grew Up with the country,2,18.02,1
...,...,...,...,...
192,Of Mice and Men,2,47.11,1
193,My Perfect Mistake (Over the Top #1),2,38.92,1
194,Meditations,2,25.89,1
195,Frankenstein,2,38.00,1
